In [1]:
import pandas as pd
from datasets import Dataset
from transformers import BartTokenizer, BartForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments

C:\Users\MSY\anaconda3\envs\pytorch_p3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df  = pd.read_csv("dataset.csv")

In [3]:
df = df.head(100000)

In [4]:
model_name = "facebook/bart-base"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

In [5]:
def preprocess_function(examples):
    inputs = examples["text"]
    targets = examples["summary"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')
    
    # 생성할 레이블을 인코딩하고, max_length로 패딩
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [6]:
dataset = Dataset.from_pandas(df)

In [7]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Parameter 'function'=<function preprocess_function at 0x00000269D998F920> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map:   0%|          | 0/100000 [00:00<?, ? examples/s]C:\Users\MSY\anaconda3\envs\pytorch_p3\Lib\site-packages\transformers\tokenization_utils_base.py:4144: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnin

In [8]:
import torch
if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs")
    model = torch.nn.DataParallel(model)

# 모델을 GPU로 이동
model = model.to("cuda")

Using 2 GPUs


In [9]:
train_test_split = tokenized_dataset.train_test_split(test_size=0.3)

train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# 데이터셋을 PyTorch 형식으로 변환
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [10]:
import os

# 로그 디렉토리 생성 및 확인
log_dir = "./results"
os.makedirs(log_dir, exist_ok=True)

if not os.path.exists(log_dir):
    raise Exception(f"Log directory {log_dir} was not created successfully")

print(f"Log directory {log_dir} is ready")

Log directory ./results is ready


In [19]:
training_args = Seq2SeqTrainingArguments(
    output_dir='/summary/results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    remove_unused_columns=False,
    logging_dir='/summary/logs',
    logging_steps=10
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)


In [ ]:
trainer.train()

C:\Users\MSY\anaconda3\envs\pytorch_p3\Lib\site-packages\transformers\models\bart\modeling_bart.py:496: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
C:\Users\MSY\anaconda3\envs\pytorch_p3\Lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
C:\Users\MSY\anaconda3\envs\pytorch_p3\Lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn("PyTorch is not compiled with NCCL support")


Epoch,Training Loss,Validation Loss
1,0.753000,No log


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
C:\Users\MSY\anaconda3\envs\pytorch_p3\Lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
C:\Users\MSY\anaconda3\envs\pytorch_p3\Lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn("PyTorch is not compiled with NCCL support")
Some non-default generation parameters are set in the model config. T

In [ ]:
trainer.evaluate()

In [ ]:
def summarize(text):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# 예시 문서 요약
example_text = "배달의민족이 독일 자본에 매각된 것을 놓고 말들이 많다.\n 민족 정서를 배반했다..."
print(summarize(example_text))

In [ ]:
trainer.save_model('./saved_model')

# 토크나이저 저장
tokenizer.save_pretrained('./saved_model')